# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [2]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv


In [12]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [3]:
# Import the necessary CSVs to Pandas DataFrames
sfo_data = pd.read_csv('sfo_neighborhoods_census_data.csv', index_col='year')

locations_df = pd.read_csv('neighborhoods_coordinates.csv')

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [26]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""

    mean_housing_units = sfo_data.groupby('year').mean('housing_units')

    housing_units_plot = mean_housing_units.plot(
        kind='bar', 
        ylim = [370000, 387500], 
        xlabel = 'Year', 
        ylabel = 'Housing Units', 
        title = 'Housing Units in San Francisco from 2010 to 2016'
    )
    return(housing_units_plot)



def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    average_rent_per_year = sfo_data.groupby('year').mean(['sale_price_sqr_foot', 'gross_rent'])

    rent_line_chart = average_rent_per_year['gross_rent'].plot(
        title='Average Gross Rent by Year', 
        xlabel='Year', 
        ylabel='Gross Rent'
    )
    return(rent_line_chart)
    


def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_sales_price = sfo_data.groupby('year').mean(['sale_price_sqr_foot', 'gross_rent'])
    
    price_line_chart = average_sales_price['sale_price_sqr_foot'].plot(
        title='Average Price per SqFt by Year', 
        xlabel='Year', 
        ylabel='Price per SqFt'
    )
    return(price_line_chart)


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    sfo_data2 = sfo_data.reset_index()
    average_hood_prices = sfo_data2.hvplot.line(
        groupby='neighborhood',
        x='year',
        y='sale_price_sqr_foot',
        xlabel='Year',
        ylabel='Average sale price for SF neighborhoods'
    )
    return(average_hood_prices)



#Top 10 most expensive neighborhoods df:
top_10_expensive_data = sfo_data.groupby('neighborhood').mean(['sale_price_sqr_foot','housing_units','gross_rent']).sort_values('sale_price_sqr_foot', ascending=False).head(10).reset_index()


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    top_10_expensive_plot = top_10_expensive_data.hvplot.bar(
        title='Top 10 most expensive neighborhoods in SF',
        x='neighborhood',
        y='sale_price_sqr_foot',
        xlabel='Neighborhood',
        ylabel='Sale price per sqr ft.',
        rot=60,
        height=500,
        width=900
    )
    return(top_10_expensive_plot)



def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    most_expensive_hoods_comparison_plot = top_10_expensive_data.hvplot.bar(
        title='Comparison of Rent and Sales Prices of Most Expensive Neighborhoods',
        x='neighborhood',
        y=['sale_price_sqr_foot', 'gross_rent'],
        xlabel='Neighborhood',
        ylabel='Amount',
        rot=60,
        height=500,
        width=900
    )
    return(most_expensive_hoods_comparison_plot)
    
    

def parallel_coordinates():
    """Parallel Coordinates Plot."""

    parallel_coordinates_plot = px.parallel_coordinates(
        top_10_expensive_data,
        title='Parallel Coordinates Analysis of Most Expensive San Francisco Neighborhoods',
        color='sale_price_sqr_foot'
    )
    return(parallel_coordinates_plot)



def parallel_categories():
    """Parallel Categories Plot."""
    
    parallel_categories_plot = px.parallel_categories(
        top_10_expensive_data,
        title='Parallel Categories Analysis of Most Expensive San Francisco Neighborhoods',
        dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
        color='sale_price_sqr_foot'
    )
    return(parallel_categories_plot)



def neighborhood_map():
    """Neighborhood Map."""

    neighborhood_values = sfo_data.groupby('neighborhood').mean().reset_index()
    hood_location_values = pd.concat([locations_df, neighborhood_values], axis='columns', join='inner')

    hood_map = px.scatter_mapbox(
        hood_location_values,
        title='Average Sale Price Per Square Foot and Gross Rent in San Francisco',
        lat='Lat',
        lon='Lon',
        size='sale_price_sqr_foot',
        color='gross_rent'
    )
    return(hood_map)   


def sunburst():
    """Sunburst Plot."""

    mean_sfo_data = sfo_data.groupby(['year', 'neighborhood']).mean('sale_price_sqr_foot')
    mean_sfo_data_df = pd.DataFrame(mean_sfo_data).reset_index()
    df_expensive_neighborhoods_per_year = mean_sfo_data_df[mean_sfo_data_df["neighborhood"].isin(top_10_expensive_data["neighborhood"])]

    sunburst_plot = px.sunburst(
        df_expensive_neighborhoods_per_year,
        title='Cost Analysis of Most Expensive Neighborhoods in San Francisco per Year',
        path=['year', 'neighborhood'],
        values='sale_price_sqr_foot',
        color='gross_rent',
        color_continuous_scale='thermal'
    )
    return(sunburst_plot)


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard

db_title = '# San Francisco Real Estate Analysis'

# Create a tab layout for the dashboard

map_tab = pn.Column(
    '## San Francisco neighborhood average rent and price per square foot values at a glance',
    neighborhood_map()
)

yearly_tab = pn.Row(
    '## Yearly housing units, average rent and price per square foot comparison',
    housing_units_per_year(),
    average_gross_rent(),
    average_sales_price()
)

hoods_tab = pn.Column(
    "## Comparing San Francisco's top 10 neighborhoods",
    most_expensive_neighborhoods_rent_sales(),
    top_most_expensive_neighborhoods()
)

parallel_tab = pn.Column(
    '## Parallel plot analyses',
    parallel_coordinates(),
    parallel_categories()
)

sunburst_tab = pn.Column(
    '## Yearly comparison for top 10 San Francisco neighborhoods',
    sunburst()
)

db_tabs = pn.Tabs(
    ('San Francisco neighborhoods overview', map_tab),
    ('Yearly evaluation', yearly_tab), 
    ('Top neighborhoods', hoods_tab),
    ('Parallel plots', parallel_tab),
    ('Sunburst', sunburst_tab)
    )
    
# Create the dashboard

sanfran_db = pn.Column(db_title, db_tabs)

## Serve the Panel Dashboard

In [28]:
# Serve the# dashboard
sanfran_db.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'93e94b0b-3d0e-4222-a6df-c97d28bde0da': {'defs': …

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Row
            [0] Markdown(str)
            [1] Str(AxesSubplot)
            [2] Str(AxesSubplot)
            [3] Str(AxesSubplot)
        [2] Column
            [0] Markdown(str)
            [1] HoloViews(Bars)
            [2] HoloViews(Bars)
        [3] Column
            [0] Markdown(str)
            [1] Plotly(Figure, visible=False)
            [2] Plotly(Figure, visible=False)
        [4] Column
            [0] Markdown(str)
            [1] Plotly(Figure, visible=False)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()



In [ ]:
# average_gross_rent()

    

In [4]:
# average_sales_price()


In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [10]:
# most_expensive_neighborhoods_rent_sales()


:Bars   [neighborhood,Variable]   (value)

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()